# Floc Analysis

This page shows an example of doing an analysis of the watercolumn floc using the Dask. The goal of this work is to understand changes in the concentration of "floc", or bacterial material that has been flushed from the hydrothermal system into the ocean. Changes in floc are an indicator of changes in the hydrothermal system, sometimes as a result of a magmatic event or seismic swarm.

This notebook uses [Dask](http://dask.pydata.org/en/latest/) to analyze a large number of frames to establish a proxy for the floc concentration, then plots this value using a two-dimensional multivariate histogram.

#### Get a list of CamHD files to process

In [ ]:
import pandas as pd
dbcamhd = pd.read_json('dbcamhd.json', orient="records", lines=True)
dbcamhd.head()

In [ ]:
filenames = list(dbcamhd.filename[(dbcamhd.deployment == 2) & (dbcamhd.frame_count > 5000) & (dbcamhd.frame_count < 30000)])
filenames.sort()
filenames[0]

#### Define the frame numbers from each file to process

In [ ]:
frame_numbers = [3841,3842,3843,3871,3872,3873,3901,3902,3903,3931,3932,
                 3933,3961,3962,3963,3991,3992,3993,4021,4022,4023,4051,
                 4052,4053,4081,4082,4083,4111,4112,4113,4141,4142,4143,
                 4171,4172,4173,4201,4202,4203,4231,4232,4233,4261,4262,
                 4263,4291,4292,4293,4321,4322,4323,4351,4352,4353,4381,
                 4382,4383,4411,4412,4413]

# NOTE: the full frame list takes ~1 hour to process here we
# just process a subset which takes about 5 minutes 
frame_numbers = [3841, 3933, 4052, 4171, 4263, 4382]

These frame numbers correspond to times when the camera system is looking over the "shoulder" of Mushroom vent. For each file in filenames, and for each frame in frame_numbers, we will process a 1024x1024 subimage that looks like this:

In [ ]:
%matplotlib inline
import pycamhd as camhd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

plt.rc('figure', figsize=(11, 11))
frame = camhd.get_frame(filenames[1], frame_numbers[0], 'rgb24')
fig, ax = plt.subplots()
im1 = ax.imshow(frame)
im1.set_cmap('gray')
plt.yticks(np.arange(0,1081,270))
plt.xticks(np.arange(0,1921,480))

rect = patches.Rectangle((10,10),1024,1024,linewidth=1.5,edgecolor='w',facecolor='none')
ax.add_patch(rect)
plt.show();

#### Show the filter that will be used to filter images in the frequency domain
To deal with variations in lighting and high-frequency noise, we filter each subimage using a Butterworth bandpass filter.

In [ ]:
def butterworth(d1, d2, n):
    x = np.arange(-1024/2+0.5,1024/2+1-0.5)
    xx, yy = np.meshgrid(x, x)
    d = np.sqrt(xx**2+yy**2)
    bff = (1 - (1./(1 + (d/d1)**(2*n))))*(1/(1 + (d/d2)**(2*n)))
    return bff

In [ ]:
d1 = 20 # low cut wavenumber
d2 = 400 # high cut wavenumber
n = 4
bff = butterworth(d1, d2, n)
plt.rc('figure', figsize=(6, 6))
imgplot = plt.imshow(bff, cmap='gray')

#### Define the floc proxy function
The floc proxy is simply the number of pixels in each filtered subimage that have a value greater than 4000.

In [ ]:
def calc_floc_proxy(frame, bff):
    I = frame[0:1024, 0:1024]
    I_fft = np.fft.fft2(I)
    I_fft_shift = np.fft.fftshift(I_fft)
    I_fft_shift_filt = I_fft_shift*bff # filter with the Butterworth filter
    I_fft_filt = np.fft.ifftshift(I_fft_shift_filt)
    I_filt = np.fft.ifft2(I_fft_filt)
    return (np.absolute(I_filt)>4000).sum()

#### Start a Dask cluster

In [ ]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=32)
cluster

In [ ]:
from dask.distributed import Client
client = Client(cluster)
client

#### Calculate the floc_proxy using Dask parallelization

In [ ]:
from dask import delayed
delayed_floc_proxy = []
bff = delayed(butterworth)(d1, d2, n)
for filename in filenames:
    moov_atom = delayed(camhd.get_moov_atom)(filename)                    
    for frame_number in frame_numbers:
        frame = delayed(camhd.get_frame)(filename, frame_number, 'gray16le', moov_atom)
        delayed_floc_proxy.append(delayed(calc_floc_proxy)(frame, bff))

In [ ]:
%%time
from dask import compute
floc_proxy = compute(*delayed_floc_proxy)

#### Get a timestamp for each frame

In [ ]:
import datetime, math
import matplotlib.dates as dates
frame_timestamp = []
for filename in filenames:
    timestamp = dbcamhd['timestamp'][dbcamhd.filename == filename].iloc[0]
    for frame_number in frame_numbers:
        timestamp = timestamp + frame_number/29.97
        dt = datetime.datetime.fromtimestamp(timestamp)
        frame_timestamp.append(dates.date2num(dt))

#### Plot a two-dimensional multivariate histogram of the results

In [ ]:
%config InlineBackend.figure_format = 'svg'
plt.rc('font', size=11)
fig, ax = plt.subplots()
fig.set_size_inches(14, 6)
fig.frameon = False
hb1 = ax.hexbin(frame_timestamp, floc_proxy, vmin=0, vmax=1, bins='log', linewidths=0.25,
  gridsize=(225,4500), mincnt=1, cmap=plt.cm.BuPu)
fig.colorbar(hb1)
ax.set_ylim([0, 8000])
ax.set_xlim([frame_timestamp[0],frame_timestamp[-1]])
ax.yaxis.grid(True)
ax.xaxis.grid(True)
months = dates.MonthLocator()  # every month
monthsFmt = dates.DateFormatter('%b %Y')
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(monthsFmt)
plt.ylabel('Floc Proxy Value');

Starting in mid-June a large "floc event" occurs where the floc proxy values increase on average by about a factor of ten. The cause of this floc event is being investigated.

### References

PyCamHD: https://github.com/tjcrone/pycamhd<br>
Raw Data Archive: https://rawdata.oceanobservatories.org/files/RS03ASHS/PN03B/06-CAMHDA301/<br>
AGU Abstract: https://agu.confex.com/agu/fm16/meetingapp.cgi/Paper/192670<br>
AGU Poster: https://drive.google.com/open?id=0B-dWW4GM434obGpTM0FZME10Nkk<br>
Dask: http://dask.pydata.org/en/latest/